# Test

In [1]:
%load_ext autoreload
%autoreload 2

Import datasets using functions from data/data.py.

Run ```pip install -e .``` if module importing isn't working.

In [4]:
from src.data.data import get_in_domain, get_out_domain

in_domain = get_in_domain()
out_domain = get_out_domain()

in_domain

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})